In [ ]:
from env import *
from swayam import *

### Tool Calling with Swayam Tools

Let's now start to link the functionality of an LLM Agent with a locally available function calls.

In [ ]:
agent = Swayam.agent(run_id="test")

#### Let's see what happens without a function call

In [ ]:
agent.execute(Prompt.text("What's the weather like in Boston?"))

### Some Dummy Functions

Let's create some dummy functions

In [ ]:
from enum import Enum
class UnitEnum(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit):
    import random
    i = random.randint(-10, 40)
    if unit == UnitEnum.CELSIUS:
        return f"{i}°C"
    elif unit == UnitEnum.FAHRENHEIT:
        return f"{i*9/5 + 32}°F"
    else:
        raise ValueError("Invalid unit")
    
def calculate(expression):
    return eval(expression)

def search_email(text):
    import re
    emails = re.findall(r'([\w\.-]+@[\w\.-]+)', text)
    if emails:
        return emails
    else:
        return None

## Tool Creation

As a part of an LLM workflow, we want to trigger the above local function based on response from the LLM (using function calling).

Before we can do that, we need to create a sort of bridge between a function from the view point of an LLM vs what it means in Python.

Here we see how we can create convert a function into a Tool.

### Let's create the structures needed by the Tools

In [ ]:
from pydantic import BaseModel, Field

class WeatherInfoModel(BaseModel):
    location: str = Field(..., title="Location", description="Location to get weather information for")
    unit: UnitEnum = Field(UnitEnum.CELSIUS, title="Unit", description="Unit of temperature")
    
class CalculationModel(BaseModel):
    expression: str = Field(..., title="Expression", description="Mathematical expression to evaluate")
    
class EmailSearchModel(BaseModel):
    text: str = Field(..., title="Text", description="Text to search for emails")
    
WeatherInfoStructure = Structure.build(WeatherInfoModel)
CalculationStructure = Structure.build(CalculationModel)
EmailSearchStructure = Structure.build(EmailSearchModel)

### Create Tools 

We can use **Tool.build** helper function to create a tool.

In [ ]:
from swayam import Tool, Structure

WeatherInfo = Tool.build("WeatherInfo", 
                         target=get_current_weather, 
                         desc="Get the current weather in a given location",
                         call_structure=WeatherInfoStructure
)

Calculator = Tool.build("Calculator",
                        target=calculate,
                        desc="Evaluates a mathematical expression",
                        call_structure=CalculationStructure
)

EmailSearch = Tool.build("EmailSearch",
                        target=search_email,
                        desc="Searches email addresses in a given text",
                        call_structure=EmailSearchStructure
)

### Tool Execution

The key thing to keep in mind is that all arguments are passed as keyword arguments irrespective of the underlying function.

In [ ]:
WeatherInfo(location="dummy")

In [ ]:
WeatherInfo(**{"location": "dummy"})

In [ ]:
WeatherInfo(**{"location": "dummy", "unit":"fahrenheit"})

In [ ]:
Calculator(expression="2 + 5")

In [ ]:
EmailSearch(text="This is a sample text with a@b.com address")

In [ ]:
EmailSearch(text="This is a sample text with no email address")

### Tool Definition

One of the key benefits of creating a Tool is that it self describes itself, in a way compatible with an LLM.

In [ ]:
print(WeatherInfo.definition)

In [ ]:
print(Calculator.definition)

In [ ]:
print(EmailSearch.definition)

Let's create the list of tool definitions to be shared with the LLM.

In [ ]:
tools = [WeatherInfo, Calculator, EmailSearch]

In [ ]:
user_prompt = Prompt.text("What's the weather like in Boston?", tools=tools)
agent.execute(user_prompt)

In [ ]:
agent.execute(Prompt.text("What is 7 *30 + 13?", tools=tools))

In [ ]:
text = "This is a sample text with a@b.com and c@d.com addresses"
prompt_text = f"""Find whether there is an email address in the following text marked with triple backticks 
            ```{text}```
            """
agent.execute(Prompt.text(prompt_text, tools=tools))